In [15]:
import pandas as pd
import numpy as np
import re
import ast
from tqdm import tqdm

In [43]:
movie_lines_path = "/Users/rohan/Desktop/projects/python-chatbot/movie_lines.txt"
movie_conversations_path = "/Users/rohan/Desktop/projects/python-chatbot/movie_conversations.txt"
movie_conversation_fields = ['char1ID', 'char2ID', 'movieID', 'utterances']
movie_lines_fields = ['lineId', 'charId', 'movieId', 'charname', 'text']

def splitConversations(max_len=60, fast_preprocessing=True):
    conversations = getconv()
    data = []
    for i, conversation in enumerate(tqdm(conversations)):
        lines = conversation['lines']
        for i in range(len(lines) - 1):
            request = preprocess_text(lines[i])
            reply = preprocess_text(lines[i + 1])
            if 0 < len(request) <= max_len and 0 < len(reply) <= max_len:
                data += [(request, reply)]
    return data

def getlines():
    with open(movie_lines_path, 'r', encoding='iso-8859-1') as f:
        lines = {}
        for line in f:
            values = line.split(' +++$+++ ')
            lineobj = {}
            for i, field in enumerate(movie_lines_fields):
                lineobj[field] = values[i]
                #lineobj['id'] = int(re.sub('L','',lineobj['lineId']))
            lines[lineobj['lineId']] = lineobj
    return(lines)

def getconv():
    lines = getlines()
    with open(movie_conversations_path, 'r', encoding='iso-8859-1') as f:
        conversations = []
        for line in f:
            values = line.split(' +++$+++ ')
            #print(len(values))
            lineobj = {}
            for i, field in enumerate(movie_conversation_fields):
                lineobj[field] = values[i]
            lineIds = ast.literal_eval(lineobj["utterances"])
            lineobj['lines'] = []
            for lineid in lineIds:
                lineobj['lines'].append(lines[lineid]['text'])
            conversations.append(lineobj)
    return conversations

def preprocess_text(line):
    GOOD_SYMBOLS_RE = re.compile('[^0-9a-z ]')
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#+_]')
    REPLACE_SEVERAL_SPACES = re.compile('\s+')
    line = line.lower()
    line = re.sub('\n','', line)
    line = REPLACE_BY_SPACE_RE.sub(' ', line)
    line = GOOD_SYMBOLS_RE.sub('', line)
    line = REPLACE_SEVERAL_SPACES.sub(' ', line)
    return line.strip()

In [44]:
cornell_data = splitConversations()

100%|██████████| 83097/83097 [00:03<00:00, 22256.79it/s]


In [45]:
cornell_data[1:5]

[('no no its my fault we didnt have a proper introduction', 'cameron'),
 ('gosh if only we could find kat a boyfriend', 'let me see what i can do'),
 ('cesc ma tete this is my head', 'right see youre ready for the quiz'),
 ('thats because its such a nice one', 'forget french')]

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer